In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('data_small_update.csv', parse_dates = True)

In [ ]:
data.head()

# Balancing the dataset

In [4]:
from sklearn.model_selection import train_test_split as tts

In [5]:
X = data.drop(['anomaly_2', 'anomaly'],axis=1)
y = data.anomaly_2

In [6]:
np.sum(y==0),np.sum(y==1)

(8261727, 5487)

In [7]:
# split data to preserve ratio of anomalies
X_train, X_test, y_train, y_test = tts(X,y,train_size=0.75)

In [8]:
np.sum(y_test==0),np.sum(y_test==1)

(2065375, 1429)

In [9]:
# concatenate training set for resampling
train = pd.concat([X_train, y_train], axis=1)

In [10]:
# check how many rows have anomoly_2==1
train[train.anomaly_2==1]

# create new dataset to balance the training set
databal = pd.DataFrame(train.loc[train.anomaly_2==1])

# copy the anomalys 50 times 
rows_to_copy = databal[data.anomaly_2==1]
copies = [rows_to_copy]*50
databal = pd.concat(copies+[databal], ignore_index=False)

C:\Users\Jethro.Thorne\AppData\Local\Temp\ipykernel_12764\952307825.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rows_to_copy = databal[data.anomaly_2==1]


In [65]:
# check 
databal.shape

(211191, 31)

In [11]:
# sample 211191 non-anomalous rows from training data (with replacement)
samples = train.loc[train.anomaly_2==0].sample(n=211191, replace=True)

# append the samples to the dataset
databal = databal.append(samples)

C:\Users\Jethro.Thorne\AppData\Local\Temp\ipykernel_12764\3859546890.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  databal = databal.append(samples)


In [67]:
databal.shape

(422382, 31)

## Logistic Regression Classifier

In [12]:
# preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler

# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# classification model
from sklearn.linear_model import LogisticRegression as logreg

# grid search 
from sklearn.model_selection import GridSearchCV, train_test_split as tts

# metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

In [13]:
X = databal.drop(['day','anomaly_2','dns_qname','dns_qdomainname','timefirstseen','timelastseen','domain','created','updated','expires'],axis=1)
y = databal['anomaly_2']

In [14]:
np.sum(y_test==0)

2065375

In [73]:
X

,Unnamed: 0,shieldid,num_clients,num_responses,shieldid_count,rank,whois,timeseen,tld,mal_tld,...,month_firstseen,day_firstseen,hour_firstseen,month_lastseen,day_lastseen,hour_lastseen,day_created,month_created,day_expires,month_expires
8103476,8103698,fecjavfec,1,2,1,0.0,whois.tucows.com,0.000,blog,0,...,1,2,13,1,2,13,3.0,1.0,3.0,1.0
8142673,8142900,fecjavfec,2,3,1,0.0,whois.your-server.de,0.033,com,0,...,12,2,15,12,2,15,0.0,11.0,1.0,11.0
7566222,7566321,fecjavfec,1,2,1,0.0,whois.godaddy.com,4764.224,org,0,...,12,2,18,12,2,19,1.0,6.0,4.0,6.0
8099609,8099831,scefunnu,2,2,1,0.0,whois.godaddy.com,11596.613,com,0,...,12,5,17,12,5,20,1.0,2.0,6.0,2.0
7661695,7661861,tyftyiep,2,102,14,0.0,whois.godaddy.com,29382.393,com,0,...,1,1,12,1,1,21,4.0,2.0,5.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068348,3068348,thewseanwi,4,28,31,82.0,whois.markmonitor.com,81443.421,com,0,...,1,5,1,1,5,23,0.0,11.0,4.0,11.0
334946,334946,fecjavfec,2,6,45,119.0,whois.markmonitor.com,0.025,com,0,...,12,2,17,12,2,17,1.0,4.0,3.0,4.0
7596597,7596696,lakuntautt,1,4,3,0.0,whois.enterprice.net,0.000,com,0,...,12,1,2,12,1,2,0.0,3.0,3.0,3.0
5069088,5069088,shwedvervi,1,2,36,0.0,whois.godaddy.com,23665.989,net,0,...,1,2,12,1,2,18,2.0,1.0,0.0,10.0


In [15]:
num_feat = ['num_clients', 'num_responses','shieldid_count','rank','timeseen',
                   'days_existed','day_firstseen','hour_firstseen','month_firstseen', 
                   'month_lastseen', 'day_lastseen','hour_lastseen', 'day_created',
                   'month_created', 'day_expires','month_expires']
cat_feat = ['shieldid','whois','tld','mal_tld']

In [22]:
num_processor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('scaler', StandardScaler())  
])

cat_processor = Pipeline(steps=[
     ('imputer', SimpleImputer(strategy = 'constant')),
     ('encoder', OneHotEncoder(handle_unknown='ignore'))   
 ])

In [23]:
feat_processor = ColumnTransformer(transformers = [
    ('num_pipe', num_processor,num_feat),
    ('cat_pipe', cat_processor, cat_feat)
],remainder='drop')

In [24]:
pipe = Pipeline(steps=[
    ('feature_processor', feat_processor),
    ('clf', logreg())
])
pipe

Pipeline(steps=[('feature_processor',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['num_clients',
                                                   'num_responses',
                                                   'shieldid_count', 'rank',
                                                   'timeseen', 'days_existed',
                                                   'day_firstseen',
                                                   'hour_firstseen',
                                                   'month_firstseen',
                                                   'mont

In [25]:
pipe.fit(X,y)

C:\Users\Jethro.Thorne\Downloads\Python\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('feature_processor',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['num_clients',
                                                   'num_responses',
                                                   'shieldid_count', 'rank',
                                                   'timeseen', 'days_existed',
                                                   'day_firstseen',
                                                   'hour_firstseen',
                                                   'month_firstseen',
                                                   'mont

In [26]:
np.sum(y_test)

1429

In [27]:
ypred=pipe.predict(X_test)
yproba=pipe.predict_proba(X_test)

In [28]:
confusion_matrix(y_test, ypred, labels=None, sample_weight=None, normalize=None)

array([[1767489,  297886],
       [     77,    1352]], dtype=int64)

In [29]:
np.sum(y_test==0), np.sum(y_test==1)

(2065375, 1429)